In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

from scipy.optimize import curve_fit 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
savePath = "../plot_files/throw_stat_univ/"


In [ ]:
rockbox_cv =[0, 1, 0, 0, 2, 2, 12, 10, 16, 18, 7, 6, 4, 0, 1, 0, 0, 0, 0]
ncpi0_cv = [0, 0, 3, 3, 30, 76, 143, 240, 321, 304, 256, 132, 70, 20, 16, 6, 5, 5, 1]

name_type = "ncpi0_"
cv =ncpi0_cv

#name_type = "rockbox_"
#cv = rockbox_cv

In [ ]:
cv = np.array(cv)
print(cv)

cv_plot = np.insert(cv, 0, 0)
print(cv_plot)

bins = np.arange(0, 20, 1)
print(bins)
bins_mid = np.convolve(bins, [0.5, 0.5], "valid")
print(bins_mid)

In [ ]:
half_width = 18.936 /2 

def gaus_fixedmean(X, A, sigma, C):
    return A * np.exp( -( X - half_width )**2 / ( 2 * sigma**2 ) ) + C

In [ ]:
prediction = [max(cv), 1.308,  0]

lower_bound = [-np.inf,-np.inf, 0]

upper_bound = [np.inf, np.inf, np.inf]

param_bounds=(lower_bound, upper_bound)

parameters, covariance = curve_fit(gaus_fixedmean, bins_mid, cv, p0 = prediction, bounds=param_bounds) 
err = np.sqrt(np.diag(covariance))

In [ ]:
fit_A = parameters[0] 
fit_sigma = parameters[1]
fit_C = parameters[2]

err_A = err[0]
err_sigma = err[1]
err_C = err[2]
  
fit_y = gaus_fixedmean(bins_mid, fit_A, fit_sigma, fit_C)

print("amplitude = {} +- {}".format(fit_A, err_A))
print("sigma = {} +- {}".format(fit_sigma, err_sigma))
print("offset = {} +- {}".format(fit_C, err_C))


plt.step(bins, cv_plot, label = 'MC')
plt.plot(bins_mid, fit_y, '-', label='fit') 
#plt.errorbar(bins_mid, fit_y, yerr= err_A, xerr= None)

plt.ylim(0, 10)
plt.legend()

plt.savefig(savePath+name_type+"_fit.png", dpi=200)

In [ ]:
#make a gaussian distribution for each parameter

n_univ = 500


amplitude_arr = np.random.normal(fit_A, err_A*1, n_univ)
sigma_arr = np.random.normal(fit_sigma, 0, n_univ)
C_arr = np.random.normal(fit_C, 1, n_univ)
C_arr = abs(C_arr)


fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (18,4))

ax1.hist(amplitude_arr, label =  'amplitude')
ax1.legend()

ax2.hist(sigma_arr, label =  'sigma')
ax2.legend()

ax3.hist(C_arr, label =  'offset')
ax3.legend()

plt.savefig(savePath+name_type+str("parameter_distribution.png"), dpi=200)

In [ ]:
len_univ = np.arange(0, n_univ)

universes = []

for u in len_univ:
    fit = gaus_fixedmean(bins_mid, amplitude_arr[u], sigma_arr[u], C_arr[u])
    plt.plot(bins_mid, fit, '-', label='fit')
    #plt.ylim(0, 40)
    #plt.show()
    
    universes.append(fit)
    
#make universe into array
universes = np.array(universes)
universes = np.sort(universes, axis = 0)

#construct covariance matrix
cov = np.cov(np.subtract(cv[np.newaxis,:], universes).transpose())

#error = sqrt(diag)
cov_err = np.sqrt(np.diagonal(cov))


plt.savefig(savePath+name_type+str("universe_distribution.png"), dpi=200)

In [ ]:
cv_err= np.sqrt(cv)

In [ ]:
new_cv = cv.copy()
new_cv[new_cv==0] = 1

new_cv_plot = np.insert(new_cv, 0, 0)
new_cv_err = np.sqrt(new_cv)

In [ ]:
print(cv_err)
print(cov_err)
print(new_cv_err)

In [ ]:
#-----------------------------------------------------------------#
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (12,4), sharey = True)

ax1.step(bins, cv_plot 
         , color = 'red'
        )

ax2.step(bins, new_cv_plot 
         , color = 'blue'
        )

ax3.step(bins, cv_plot
         , color = 'green'
        )
#-----------------------------------------------------------------#
#stats
stat_err = cv_err

bottom = cv - stat_err
peak = cv + stat_err
height = peak - bottom

plt. rcParams["hatch.color"] = "gray"
ax1.bar(
        x = bins[:-1]
        , height= height
        , width = np.diff(bins)
        , bottom = bottom
        , align = 'edge'
        , hatch='///'
        , fc='gray'
        , alpha = 0.2 
        , label = "square root entries"
            )
#-----------------------------------------------------------------#
#stats
stat_err = new_cv_err

bottom = new_cv - stat_err
peak = new_cv + stat_err

height = peak - bottom

plt. rcParams["hatch.color"] = "gray"
ax2.bar(
        x = bins[:-1]
        , height= height
        , width = np.diff(bins)
        , bottom = bottom
        , align = 'edge'
        , hatch='///'
        , fc='gray'
        , alpha = 0.2 
        , label = 'fill empty bin = 1'
            )
#-----------------------------------------------------------------#
#stats
stat_err = cov_err

bottom = cv - stat_err
peak = cv + stat_err

height = peak - bottom

plt. rcParams["hatch.color"] = "gray"
ax3.bar(
        x = bins[:-1]
        , height= height
        , width = np.diff(bins)
        , bottom = bottom
        , align = 'edge'
        , hatch='///'
        , fc='gray'
        , alpha = 0.2 
        , label = "universes"
            )

#-----------------------------------------------------------------#
ax1.legend()
ax2.legend()
ax3.legend()

ax1.set_ylabel("Entries")

#ax1.set_ylim(0, 30)

plt.savefig(savePath+name_type+str("modt_compare.png"), dpi=200)

In [ ]:
#-----------------------------------------------------------------#
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (12,4),sharey = True)

ax1.step(bins, np.insert(cv_err,0,0) 
         , color = 'red'
         , label = 'square root entries'
        )


ax2.step(bins, np.insert(new_cv_err,0,0) 
         , color = 'blue'
         , label = 'fill empty bin = 1'
        )

ax3.step(bins, np.insert(cov_err,0,0) 
         , color = 'green'
         , label = 'universe'
        )

ax1.set_ylabel("Statistics  Uncertainty")

ax1.legend()
ax2.legend()
ax3.legend()

#ax1.set_ylim(0, 8)

plt.savefig(savePath+name_type+str("error_compare.png"), dpi=200)

In [ ]:
print(cov_err/new_cv)

In [ ]:
#-----------------------------------------------------------------#
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (12,4), sharey = True)

ax1.step(bins, np.insert(cv_err/new_cv,0,0) 
         , color = 'red'
         , label = 'square root entries'
        )

ax2.step(bins, np.insert(new_cv_err/new_cv,0,0) 
         , color = 'blue'
         , label = 'fill empty bin = 1'
        )

ax3.step(bins, np.insert(cov_err/new_cv,0,0) 
         , color = 'green'
         , label = 'universe'
        )

ax1.legend()
ax2.legend()
ax3.legend()

#ax1.set_ylim(0, 1.5)
ax1.set_ylabel("Fractional Stat Error")
plt.savefig(savePath+name_type+str("frac_error_compare.png"), dpi=200)